# Coordinate da Indirizzo

In [16]:
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent = "myapp")
location = locator.geocode("Via Novegro 20090 Segrate")
start_lat=location.latitude 
start_lng=location.longitude
print(start_lat,start_lng)


45.4688535 9.2776371


In [17]:
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent = "myapp")
location = locator.geocode("Via Como 9 Barzago")
end_lat=location.latitude 
end_lng=location.longitude
print(end_lat,end_lng)

45.7578372 9.3097403


# Calcolo percorso più veloce

In [39]:
#Find Distance
import osmnx as ox
import networkx as nx
import numpy as np
from IPython.display import IFrame
import folium
%matplotlib inline

np.random.seed(0)
ox.__version__

ox.settings.log_console=True
place = "Milan,Lombardy,Italy"
mode="drive"
G = ox.graph_from_place(place, network_type=mode)
Gp = ox.project_graph(G)

In [ ]:
import folium
map = folium.Map(location=[start_lat, start_lng], tiles="OpenStreetMap", zoom_start=9)
map

In [34]:
#start_lat,start_lng,end_lat,end_lng
start_node = ox.nearest_nodes(Gp, start_lat, start_lng)
end_node = ox.nearest_nodes(Gp, end_lat, end_lng)

In [35]:
#start_lat,start_lng,end_lat,end_lng
start_edge = ox.nearest_edges(Gp, start_lat, start_lng)
end_edge = ox.nearest_edges(Gp, end_lat, end_lng)

In [36]:
orig = list(G)[start_node]
dest = list(G)[end_edge]
route = ox.shortest_path(G, orig, dest, weight="length")
#AD Speed
# impute speed on all edges missing data
G = ox.add_edge_speeds(G)
# calculate travel time (seconds) for all edges
G = ox.add_edge_travel_times(G)

IndexError: list index out of range

In [5]:

# see mean speed/time values by road type
edges = ox.graph_to_gdfs(G, nodes=False)
edges["highway"] = edges["highway"].astype(str)
edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)

# same thing again, but this time pass in a few default speed values (km/hour)
# to fill in edges with missing `maxspeed` from OSM
hwy_speeds = {"residential": 35, "secondary": 50, "tertiary": 60}
G = ox.add_edge_speeds(G, hwy_speeds)
G = ox.add_edge_travel_times(G)

# calculate two routes by minimizing travel distance vs travel time
orig = list(G)[1]
dest = list(G)[120]
route1 = ox.shortest_path(G, orig, dest, weight="length")
route2 = ox.shortest_path(G, orig, dest, weight="travel_time")

# compare the two routes
route1_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "length")))
route2_length = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "length")))
route1_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route1, "travel_time")))
route2_time = int(sum(ox.utils_graph.get_route_edge_attributes(G, route2, "travel_time")))
print("Route 1 is", route1_length, "meters and takes", route1_time, "seconds.")
print("Route 2 is", route2_length, "meters and takes", route2_time, "seconds.")


Route 1 is 6992 meters and takes 534 seconds.
Route 2 is 7178 meters and takes 485 seconds.


In [10]:
shortest_route_map = ox.plot_route_folium(G, route1,tiles='openstreetmap')
shortest_route_map